In [1]:
import psycopg2
import pandas as pd
from sql_queries import *
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.io import output_notebook, push_notebook, show

output_notebook()

Loading BokehJS ...

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
#query
cur.execute(play_location)
results = cur.fetchall()
#parse data
location = []
play = []
for i in range(0,len(results)):
    location.append(results[i][0])
    play.append(results[i][1])

#plot results
source = ColumnDataSource(data=dict(location=location, play=play))

p = figure(x_range=location, plot_height=550, toolbar_location="right", title="Play x Location - TOP 5")
p.vbar(x='location', top='play', width=0.2, source=source,
       line_color='white', 
       fill_color=factor_cmap('location', palette=Spectral6, factors=location))

p.xaxis.major_label_orientation = 0.5
p.legend.location = 'bottom_right'
p.y_range.start = 0
p.y_range.end = 1000

handle = show(p)

In [4]:
#query
cur.execute(play_os_level)
results = cur.fetchall()
#parse data
paid = []
free = []
total = []
os = []
for i in range(0,len(results)):
    os.append(results[i][1])
    if (results[i][0] == 'paid'):
        paid.append(results[i][2])
    elif (results[i][0] == 'free'):
        free.append(results[i][2])
    else:
        total.append(results[i][2])

os = list(set(os))
os.remove(None)
os.append('Total')

#plot
level = ['paid', 'free', 'total']

data = {'OS' : os,
        'paid'   : paid,
        'free'   : free,
        'total'   : total}

x = [ (os_, level_) for os_ in os for level_ in level ]
counts = sum(zip(data['paid'], data['free'], data['total']), ())

source = ColumnDataSource(data=dict(x=x, counts=counts))

p = figure(x_range=FactorRange(*x), plot_height=250, title="Play X OS X Level",
           toolbar_location="right", tools="")

p.vbar(x='x', top='counts', width=0.9, source=source)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)



